In [60]:
#import the lib i will use 
import pandas as pd# for readinf csv file and to handle data 
from pathlib import Path # to control the path and create a custom folders
import re  

In [61]:
# reading the csv and adding the pst file path
df= pd.read_csv("Nahla/emls.csv",index_col="Unnamed: 0")
pst_file=r"Nahla El Kazak.pst"

# EDA

In [62]:
# exploreing data
df.head()

,From,To
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,From:To:Date:Subject:Message-ID:Content-Type:M...,To:Date:Subject:Message-ID:Content-Type:MIME-V...
4,"From: ""Keren Batash (Hever-Project Management ...","To: ""shaym@valinor.co.il"" <shaym@valinor.co.il..."


In [63]:
df.dropna(inplace=True)

In [64]:
df.head()

,From,To
3,From:To:Date:Subject:Message-ID:Content-Type:M...,To:Date:Subject:Message-ID:Content-Type:MIME-V...
4,"From: ""Keren Batash (Hever-Project Management ...","To: ""shaym@valinor.co.il"" <shaym@valinor.co.il..."
5,"From: ""Keren Batash (Hever-Project Management ...","To: ""shaym@valinor.co.il"" <shaym@valinor.co.il..."
6,From:Reply-to:Subject:Date;b=eoWC5nTWAHBR9rH+U...,To:From:Reply-to:Subject:Date;b=eoWC5nTWAHBR9r...
7,From:MIME-Version:List-Id:List-Unsubscribe:Con...,To:Subject:From:MIME-Version:List-Id:List-Unsu...


## regex Function

In [65]:
 

# this function created to chech the pattern of the emails and mask the data based on this check
regex = '^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'  
  
def check(email):   
  
    if(re.search(regex,email)):   
       return True
    else:   
        return False 
      

In [66]:
def check_names(name):
    if "?" in name and "=" in name:
        return ""
    else:
        return name

In [67]:
# completeing the EDA 
df.drop_duplicates(inplace=True)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7247 entries, 3 to 18974
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   From    7247 non-null   object
 1   To      7247 non-null   object
dtypes: object(2)
memory usage: 169.9+ KB


In [69]:
df.head()

,From,To
3,From:To:Date:Subject:Message-ID:Content-Type:M...,To:Date:Subject:Message-ID:Content-Type:MIME-V...
4,"From: ""Keren Batash (Hever-Project Management ...","To: ""shaym@valinor.co.il"" <shaym@valinor.co.il..."
6,From:Reply-to:Subject:Date;b=eoWC5nTWAHBR9rH+U...,To:From:Reply-to:Subject:Date;b=eoWC5nTWAHBR9r...
7,From:MIME-Version:List-Id:List-Unsubscribe:Con...,To:Subject:From:MIME-Version:List-Id:List-Unsu...
8,From: Marketing <Marketing@aptec.com.eg>\r,To: Undisclosed recipients:;\r


### filtering the from col to get the emails and names

In [70]:
# the main function take the col and start to filter this col to dataframe with 
#email and names and remove all the null and remove duplication
def clean_from_side(emails):
    names=emails.str.split("<",expand=True)[0]
    emails=emails.str.split("<",expand=True)[1]
    final_emails=emails.str.split(">",expand=True)[0].str.lower()
    final_names=names.str.split(": ",expand=True)[1]
    final_df = pd.concat([final_emails, final_names], axis=1,keys=["Emails","Names"])

    final_df.dropna(inplace=True)
    final_df.drop_duplicates(inplace=True)
    mask=final_df["Emails"].apply(check)
    final_df["Names"]=final_df["Names"].apply(check_names)
    final_df=final_df[mask]
   
    final_df.reset_index(drop=True,inplace=True)
    return final_df


In [71]:
# calling the function for the to calls the from and the to cols 
final_df_from=clean_from_side(df["From"])
final_df_to=clean_from_side(df["To"])

### filtering To col to emails and names 

In [72]:
# saving the final output as csv files 
eml_out = Path(Path.cwd() / pst_file.split(" ")[0])


if not eml_out.exists():
  eml_out.mkdir()


final_df_from.to_csv(eml_out / "after_Filtering_From.csv")
# final_name_from.to_csv(eml_out / "after_Filtering_From_Names.csv")
final_df_to.to_csv(eml_out / "after_Filtering_To_emls.csv")
# final_name_To.to_csv(eml_out / "after_Filtering_To_Names.csv")